---

# Coopang

1. `from requests import get ` 로 처리 안됨, selenium 으로 다이내믹 처리



In [ ]:

from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

options = Options()

browser = webdriver.Chrome(options=options)

base_url = "https://www.coupang.com/np/search?component"
search_term = "썬크림"

browser.get(f"{base_url}=&q={search_term}&channel=user")


print(browser.page_source) # html source code 추출

In [ ]:
# --- bs4 처리 

soup = BeautifulSoup(browser.page_source, "html.parser")
search_product_list = soup.find("ul", class_="search-product-list")
products = search_product_list.find_all('li', recursive=False)


# 36개씩 보기 
# ul 태그 내부, li 태그 (1 level)의 개수 출력 
print(len(products))
print()

# product
for product in products:
    print(product)
    print()
    print()

- 이전 프로젝트에서 중간에 아예 다른 태그 가 있었는데, 지금은 같은 li태그중에 이름만 다른경우 같다.

```
# li 태그 중 class 네임에 따라 출력, # find, none 논리로 출력 

cnt = 0
for product in products:
    
    if product.find("li", class_="__ad-badge"):
        print('ad')

    
```

---

In [1]:
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

options = Options()

browser = webdriver.Chrome(options=options)

base_url = "https://www.coupang.com/np/search?component"
search_term = "썬크림"

browser.get(f"{base_url}=&q={search_term}&channel=user")

soup = BeautifulSoup(browser.page_source, "html.parser")
search_product_list = soup.find("ul", class_="search-product-list")


products = search_product_list.find_all('li', recursive=False)
print(products)

[<li class="search-product search-product__ad-badge" data-adsplatform='{"clickLogUri":"https://mercury.coupang.com/e.gif?r=QCLgp%2BALtrVpjM5B2K9U%2Fg2FQgTbF3AayhICUZeun%2BJBDDw4Dh7vVLIitx%2BCCYv0bNyz0cO6Qja0fDeedKey1iTPANeWZK1XNgu%2FeDc%2FLVR%2FkvGXoW8t0EnAOCvhndolJjKRchPaosz13SAlGBNOaMr5cCKeJ9UdXd1c53aSUt4MGlqrA9cT3yVePgVjR1CZvlB4S1k0a%2B0h%2FHtPXomLxQLFi86tHB9Pt9ahhFz%2FmwQA%2BdzM6ZQjO%2FLb6WVHXpJROzha5sJtdgn4kkV1xoOBKU6RnQMizyqj50ff%2BEoppJx2R6c%2BuImNmESXhvWmdMbY2nK49tqk%2B7ueiXIHWBjJ2e4SuLnahcKM%2FQqEunCoXAkr4Oouqo42qYiPfTu6cwthNQOkD1Z4mHtBWhdjINAtNhni7QrV6XzP8v2bdl5bfjSGZNMvymbAna6ptkC3R6qBofTJe%2B4ukaI0%2Bat2LOPCOV26dovxCCTm44tfGZ0PWcMNtCGQz0KVT%2B8PgYkRbWIe55nDZHEGYH9468twe%2FomcyxQxnIMUqroE7PfASfIDd1yG6bwdM45usggXEDHridXe5o6F8bEnxTEdilHdmyrHgV%2Fwm4pe1BK%2FbHehqqvXJy2a%2BfPjzJLbizXo%2Fnq9yyPuuN7Wz8v%2BYRRtOf2%2F85NzsWrTNjVOKGoFFPNpEZmB7gTwBY1XmkTPJDLjlNKCbs6w8tRqPKJ%2Bl7usGOjufIf2JEsQmmIQmB5u6QGj9X8SiBowGxcpErRSrLs3wQKTf%2BvHNh2YBxIQcYbsK71100hQK2ilFOAGrl%2F07sW1m0sfyMhlMLyWkm

---

## ad 제거

In [20]:
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

options = Options()

browser = webdriver.Chrome(options=options)

base_url = "https://www.coupang.com/np/search?component"
search_term = "썬크림"

browser.get(f"{base_url}=&q={search_term}&channel=user")

soup = BeautifulSoup(browser.page_source, "html.parser")
search_product_list = soup.find("ul", class_="search-product-list")

filtered_products = [ product for product in search_product_list.find_all('li', recursive=False) if 'search-product__ad-badge' not in product['class'] ]


In [23]:

# 36개중 9개 광고 제거 데이터
for product in filtered_products:
    print(product)


<li class="search-product" data-coupon-nudge-text="" data-coupon-tag-area="true" data-is-rocket="true" data-product-id="4583248357" data-vendor-item-id="4498803481" data-winner-vendor-item-id="4498803481" id="4583248357" style="height: auto;"><a class="search-product-link" data-is-soldout="" data-item-id="5620780687" data-product-id="4583248357" data-srp-log='{"group":"PRODUCT", "itemId":"5620780687", "productId":"4583248357", "vendorItemId":"4498803481", "page":"1", "listSize":"36", "isCcidEligible":false, "displayCcidBadge":false, "wowOnlyInstantDiscountRate": -1, "snsDiscountRate" : -1, "isLoyaltyMember": false }' data-vendor-item-id="4498803481" href="/vp/products/4583248357?itemId=5620780687&amp;vendorItemId=4498803481&amp;q=%EC%8D%AC%ED%81%AC%EB%A6%BC&amp;itemsCount=36&amp;searchId=148eff2d60ef4fcc8c3535737af186fc&amp;rank=1" style="height: 506px;" target="_blank"><dl class="search-product-wrap" style="height: 506px;"><dt class="image"> <img alt="본트리 베리 에센스 선블럭 SPF50+ PA++++, 50m

---

## 데이터 필터링

1. 링크

2. 네임

3. 가격

4. 추가 할인 쿠폰 여부

5. 유닛 당 가격


In [39]:
# 제품 링크

anchor = filtered_products[0].find('a')
print('https://www.coupang.com/'+anchor['href'])

https://www.coupang.com//vp/products/4583248357?itemId=5620780687&vendorItemId=4498803481&q=%EC%8D%AC%ED%81%AC%EB%A6%BC&itemsCount=36&searchId=148eff2d60ef4fcc8c3535737af186fc&rank=1


In [85]:
# 브랜드, 제품명, SPF, PA

filtered_products[10].select_one('.name')

<div class="name">닥터지 그린 마일드 업 선 플러스 SPF50+ PA++++ 35ml x 2p, 2개</div>

In [70]:
# 가격

filtered_products[0].select_one('.price-value')

'15,000'

In [65]:
# 추가 할인 쿠폰 있으면 if, 없으면, None

if filtered_products[11].select_one('.coupon-wot-nudge-ticket_left_border_amount'):
    print(True)
else:
    print('None')

None


In [72]:
# 유닛 별 가격

filtered_products[0].select_one('.unit-price')

<span class="unit-price">(<em>10</em>ml당 <em>1,500</em>원) </span>

---